In [73]:
import pandas as pd
import numpy as np
import random

In [74]:
filepath = 'https://raw.githubusercontent.com/S1mDu/LCA/main/Data/'

In [76]:
data = pd.read_excel(filepath + "final_db.xlsx")

In [101]:
data.columns

Index(['alim_grp_nom_fr', 'alim_ssgrp_nom_fr', 'alim_ssssgrp_nom_fr', 'CIQUAL',
       'alim_nom_fr', 'Energie, Règlement UE N° 1169/2011 (kJ/100 g)',
       'Energie, Règlement UE N° 1169/2011 (kcal/100 g)',
       'Energie, N x facteur Jones, avec fibres  (kJ/100 g)',
       'Energie, N x facteur Jones, avec fibres  (kcal/100 g)',
       'Eau (g/100 g)', 'Protéines, N x facteur de Jones (g/100 g)',
       'Protéines, N x 6.25 (g/100 g)', 'Glucides (g/100 g)',
       'Lipides (g/100 g)', 'Sucres (g/100 g)',
       'Fibres alimentaires (g/100 g)', 'Acides organiques (g/100 g)',
       'AG saturés (g/100 g)', 'AG monoinsaturés (g/100 g)',
       'AG polyinsaturés (g/100 g)', 'Groupe d'aliment',
       'Sous-groupe d'aliment', 'Nom du Produit en Français', 'LCI Name',
       'code saison (0 : hors saison ; 1 : de saison ; 2 : mix de consommation FR)',
       'code avion (1 : par avion)', 'Livraison', 'Préparation',
       'DQR - Note de qualité de la donnée (1 excellente ; 5 très faible)

In [77]:
df = data[['CIQUAL', 'alim_grp_nom_fr', 'Energie, Règlement UE N° 1169/2011 (kcal/100 g)','Protéines, N x facteur de Jones (g/100 g)', 'Glucides (g/100 g)', 'Lipides (g/100 g)', 'Sucres (g/100 g)', 'Fibres alimentaires (g/100 g)', 'AG saturés (g/100 g)', 'AG monoinsaturés (g/100 g)', 'AG polyinsaturés (g/100 g)']]

In [78]:
df.rename(columns={'Energie, Règlement UE N° 1169/2011 (kcal/100 g)': 'Energie',
                   'alim_grp_nom_fr': 'AlimGroup',
                   'Protéines, N x facteur de Jones (g/100 g)': 'Proteines',
                   'Glucides (g/100 g)': 'Carbs',
                   'Lipides (g/100 g)': 'Lipides',
                   'Sucres (g/100 g)': 'Sucres',
                   'Fibres alimentaires (g/100 g)': 'Fibres',
                   'AG saturés (g/100 g)': 'AGS',
                   'AG monoinsaturés (g/100 g)': 'AGM',
                   'AG polyinsaturés (g/100 g)': 'AGP'}, inplace=True)

C:\Users\simon\AppData\Local\Temp\ipykernel_24480\1162833374.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.rename(columns={'Energie, Règlement UE N° 1169/2011 (kcal/100 g)': 'Energie',


In [106]:
df['AlimGroup'].unique()

array(['Boissons', 'viandes, œufs, poissons et assimilés',
       'fruits, légumes, légumineuses et oléagineux', 'Mixed', 'Grains',
       'Desserts', 'Misc', 'Diary', 'Fats', 'Children'], dtype=object)

In [107]:
df['AlimGroup'] = df['AlimGroup'].replace({
    'eaux et autres boissons': 'Boissons',
    'viandes, œufs, poissons et assimilés': 'Proteines',
    'fruits, légumes, légumineuses et oléagineux': 'Vegetables',
    'entrées et plats composés': 'Mixed',
    'produits céréaliers': 'Grains',
    'produits sucrés': 'Desserts',
    'aides culinaires et ingrédients divers': 'Misc',
    'produits laitiers et assimilés': 'Diary',
    'matières grasses': 'Fats',
    'aliments infantiles': 'Children',
    'glaces et sorbets': 'Desserts',
})

C:\Users\simon\AppData\Local\Temp\ipykernel_24480\1475301774.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['AlimGroup'] = df['AlimGroup'].replace({


In [108]:
df.tail()

,CIQUAL,AlimGroup,Energie,Proteines,Carbs,Lipides,Sucres,Fibres,AGS,AGM,AGP
545,39503,Desserts,328.0,3.42,33.40,19.80,27.9,1.23,13.30,5.050,0.88
546,39516,Desserts,262.0,2.71,26.20,16.10,21.9,0.94,13.30,1.500,0.41
547,39519,Desserts,196.0,2.74,23.50,9.57,21.0,2.76,5.19,3.310,0.58
548,39529,Desserts,263.0,4.93,25.30,15.20,20.5,2.76,11.00,0.026,2.19
549,40106,Proteines,120.0,15.50,5.64,3.40,0.0,0.00,1.31,0.690,1.19


In [109]:
required_groups = ['Boissons', 'Proteines', 'Vegetables', 'Grains', 'Desserts', 'Diary', 'Fats']

In [110]:
# Ensure the group names in required_groups match the values in data['alim_grp_nom_fr']
group_indices = {group: df[df['AlimGroup'] == group].index.tolist() for group in required_groups}

# Check if any group has no items
for group, indices in group_indices.items():
    if not indices:
        print(f"Warning: Group '{group}' has no items in the dataset.")

# Initialize a list to store the plates
plates = []

# Generate 200 random plates
for _ in range(10000):
    plate = {}
    for group in required_groups:
        if group_indices[group]:  # Ensure the group has items
            selected_index = random.choice(group_indices[group])
            plate[group] = df.loc[selected_index, 'CIQUAL']  # Store the CIQUAL number
        else:
            plate[group] = None  # Handle cases where a group has no items
    plates.append(plate)

# Convert the plates list to a DataFrame
plates_df = pd.DataFrame(plates)

# Display the first few rows of the plates DataFrame
print(plates_df.head())

   Boissons  Proteines  Vegetables  Grains  Desserts  Diary   Fats
0      1003      22010       13019    9322     32028  19023  17190
1     18068       8704       15049    9310      7602  39216  16560
2      2061       6256       13001    7115     31000  23474  16570
3      2074       6256       13016    9322     23032  19693  16733
4      1005       1030       20170    9111     32028  19027  17420


In [111]:
# Create a mapping dictionary from CIQUAL to alim_nom_fr
ciqual_to_name = data.set_index('CIQUAL')['alim_nom_fr'].to_dict()

# Replace the CIQUAL numbers in plates_df with their corresponding full names
named_plates_df = plates_df.replace(ciqual_to_name)

named_plates_df.head(3)

,Boissons,Proteines,Vegetables,Grains,Desserts,Diary,Fats
0,Liqueur,"Oeuf, dur","Groseille, crue","Orge perlée, bouilli/cuite à l'eau, non salée","Céréales pour petit déjeuner ""équilibre"" au ch...","Lait entier, UHT",Huile de maïs
1,"Cola, non sucré, avec édulcorants, sans caféine","Boudin noir, sauté/poêlé",Mélange apéritif de graines (non salées) et ra...,"Avoine, crue","Croissant, sans précision","Clafoutis aux fruits, préemballé",Graisse d'oie
2,"Nectar multifruit, standard","Boeuf, steak haché 20% MG, cru","Abricot, dénoyauté, sec",Pain au son,Barre chocolatée biscuitée,Profiteroles (crème pâtissière et sauce chocol...,Graisse de dinde


In [112]:
nutrient_columns = ['Energie', 'Proteines', 'Carbs', 'Lipides', 'Sucres', 'Fibres', 'AGS', 'AGM', 'AGP']
# Initialize new columns in plates_df for the nutrient sums
for nutrient in nutrient_columns:
    plates_df[f'tot_{nutrient.lower()}'] = 0

# Calculate the nutrient sums for each row in plates_df
for index, row in plates_df.iterrows():
    nutrient_sums = df[df['CIQUAL'].isin(row.values)][nutrient_columns].sum()
    for nutrient in nutrient_columns:
        plates_df.at[index, f'tot_{nutrient.lower()}'] = nutrient_sums[nutrient]

# Display the updated plates_df
plates_df.head()

C:\Users\simon\AppData\Local\Temp\ipykernel_24480\2195822664.py:10: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '2025.6' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  plates_df.at[index, f'tot_{nutrient.lower()}'] = nutrient_sums[nutrient]
C:\Users\simon\AppData\Local\Temp\ipykernel_24480\2195822664.py:10: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '31.040000000000003' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  plates_df.at[index, f'tot_{nutrient.lower()}'] = nutrient_sums[nutrient]
C:\Users\simon\AppData\Local\Temp\ipykernel_24480\2195822664.py:10: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '151.02999999999997' has dtype incompatible with int64, pl

,Boissons,Proteines,Vegetables,Grains,Desserts,Diary,Fats,tot_energie,tot_proteines,tot_carbs,tot_lipides,tot_sucres,tot_fibres,tot_ags,tot_agm,tot_agp
0,1003,22010,13019,9322,32028,19023,17190,2025.60,31.04,151.030,120.700,69.830,12.46,21.606,34.467,57.610
1,18068,8704,15049,9310,7602,39216,16560,2630.87,53.17,159.080,190.710,63.890,24.40,56.840,98.770,25.120
2,2061,6256,13001,7115,31000,23474,16570,2480.70,40.16,209.800,163.289,115.140,19.05,62.823,61.620,26.380
3,2074,6256,13016,9322,23032,19693,16733,1386.60,33.00,102.350,89.480,57.380,15.03,33.709,35.963,13.489
4,1005,1030,20170,9111,32028,19027,17420,2277.10,52.90,161.666,126.720,83.896,11.44,24.964,35.417,60.950


In [113]:
# Randomly select 21 plates from the selected_meals DataFrame
random_plates = plates_df.sample(n=21, random_state=42)

# Define the nutrient columns to sum
nutrient_columns = ['tot_energie', 'tot_proteines', 'tot_carbs', 'tot_lipides', 'tot_sucres', 'tot_fibres', 'tot_ags', 'tot_agm', 'tot_agp']

# Calculate the total nutrients for the selected 21 plates
total_nutrients = random_plates[nutrient_columns].sum()

# Display the selected plates and their total nutrients
print("Selected Plates:")
display(random_plates.head())
print(random_plates.shape)
print("\nTotal Nutrients for the Week:")
print(total_nutrients)

Selected Plates:


,Boissons,Proteines,Vegetables,Grains,Desserts,Diary,Fats,tot_energie,tot_proteines,tot_carbs,tot_lipides,tot_sucres,tot_fibres,tot_ags,tot_agm,tot_agp
6252,5004,8331,13046,38104,31042,19027,16550,3074.2,30.18,256.46,210.270,190.31,11.10,72.550,99.90,29.85
4684,1000,8211,4015,7180,23881,19201,17190,2316.8,35.66,124.39,155.090,29.03,6.18,29.830,54.10,62.37
1731,2061,21507,13018,9390,32133,19693,17640,1990.0,45.73,174.15,117.759,52.97,20.50,34.883,39.47,36.65
4742,18901,22508,13004,9870,7711,19550,16060,2055.3,37.65,130.08,149.750,19.58,12.95,102.540,26.74,10.34
4521,18903,25224,13001,7425,7710,19037,17100,2214.4,46.52,194.38,135.520,57.19,22.09,29.280,79.68,17.75


(21, 16)

Total Nutrients for the Week:
tot_energie      48569.6000
tot_proteines     1021.2500
tot_carbs         3546.3800
tot_lipides       3093.1940
tot_sucres        1426.1600
tot_fibres         360.9780
tot_ags            952.1250
tot_agm           1230.7629
tot_agp            723.7417
dtype: float64


In [114]:
plates_df.sample(n=21)[['Boissons', 'Proteines', 'Vegetables', 'Grains', 'Desserts', 'Diary', 'Fats']].values.tolist()

[[18901, 26090, 13021, 7525, 7712, 12300, 16550],
 [18060, 21500, 13029, 9811, 32123, 19661, 16030],
 [18041, 26029, 20116, 7210, 23880, 19508, 17190],
 [1005, 26096, 15018, 7403, 7733, 12320, 17700],
 [18100, 10009, 15018, 7201, 32021, 19575, 17100],
 [18901, 28300, 15049, 9821, 24615, 19023, 17420],
 [2061, 10023, 20587, 7004, 32029, 19041, 16570],
 [2073, 25598, 20500, 9101, 24056, 19575, 17630],
 [18066, 21515, 15049, 9322, 7711, 19661, 17650],
 [1004, 26013, 20016, 9060, 32016, 19698, 17100],
 [2074, 8933, 20047, 7257, 7710, 19689, 16550],
 [18060, 28004, 13029, 38403, 31001, 19201, 16560],
 [1001, 26034, 20116, 38107, 23884, 19037, 17400],
 [2073, 28401, 13021, 7352, 31101, 19589, 17210],
 [5214, 6202, 20502, 7115, 31106, 20911, 17190],
 [1015, 8015, 20047, 7170, 31113, 19590, 17630],
 [2012, 22000, 20500, 9345, 23884, 39220, 17700],
 [18154, 6257, 4039, 9360, 7620, 39216, 16030],
 [1003, 21513, 15201, 7180, 7733, 19021, 17700],
 [2000, 26016, 20016, 9390, 32001, 19590, 16540],
 

In [115]:
# Initialize a list to store the results for each week
weeks_data = []

# Perform the operation 10 times
for i in range(10000):
    # Randomly select 21 plates from the selected_meals DataFrame
    random_plates = plates_df.sample(n=21, random_state=i)  # Use a different random_state for each iteration

    # Define the nutrient columns to sum
    nutrient_columns = ['tot_energie', 'tot_proteines', 'tot_carbs', 'tot_lipides', 'tot_sucres', 'tot_fibres', 'tot_ags', 'tot_agm', 'tot_agp']

    # Calculate the total nutrients for the selected 21 plates
    total_nutrients = random_plates[nutrient_columns].sum()

    # Retrieve the detailed composition (CIQUAL numbers) for each plate
    detailed_composition = random_plates[['Boissons', 'Proteines', 'Vegetables', 'Grains', 'Desserts', 'Diary', 'Fats']].values.tolist()  # Each row contains the CIQUAL numbers for a plate

    # Store the results in a dictionary
    week_result = {'week': i + 1}
    week_result.update(total_nutrients.to_dict())  # Add nutrient totals to the dictionary
    week_result['composition'] = detailed_composition  # Add the detailed composition

    # Append the result to the list
    weeks_data.append(week_result)

# Convert the list of results to a DataFrame for comparison
weeks_df = pd.DataFrame(weeks_data)

# Display the results
display(weeks_df.head())

,week,tot_energie,tot_proteines,tot_carbs,tot_lipides,tot_sucres,tot_fibres,tot_ags,tot_agm,tot_agp,composition
0,1,49846.00,1106.440,3503.950,3235.833,1385.200,409.160,1095.040,1302.5631,616.3440,"[[18041, 6130, 20901, 7525, 31001, 19575, 1763..."
1,2,46055.66,1035.930,2961.016,3024.838,1144.936,287.706,1064.147,1246.2573,521.4967,"[[18154, 26232, 20502, 7180, 7720, 19689, 1652..."
2,3,44910.77,1135.060,3237.246,2842.862,1137.056,312.770,914.581,1150.0271,594.9914,"[[2061, 6130, 15018, 9321, 23472, 19693, 17210..."
3,4,44234.73,1070.522,3035.660,2853.001,1054.620,393.294,937.390,1053.3627,686.1777,"[[18066, 30005, 20016, 7115, 24041, 19041, 165..."
4,5,48846.86,1092.933,3582.186,3134.699,1490.176,360.509,1223.817,1152.2319,562.9187,"[[5214, 6253, 20062, 7258, 32018, 19698, 17170..."


In [129]:
# Define nutrient needs (example values, replace with actual needs)
healthy_needs = {
    'Energie': 18894,
    'Proteines': 392,
    'Carbs': 2800,
    'Lipides': 630,
    'Sucres': 1,
    'Fibres': 280,
    'AGS': 1,
    'AGM': 1,
    'AGP': 1
}

cheap_needs = {
    'Energie': 1567*7,
    'Proteines': 55.34*7,
    'Carbs': 196.71*7,
    'Lipides': 59.14*7,
    'Sucres': 85.4*7,
    'Fibres': 10.8*7,
    'AGS': 19.977*7,
    'AGM': 18.915*7,
    'AGP': 12.851*7
}

In [130]:
healthy_selected_diet = weeks_df[
    (weeks_df['tot_energie'] >= healthy_needs['Energie']) &
    (weeks_df['tot_proteines'] >= healthy_needs['Proteines']) &
    (weeks_df['tot_carbs'] >= healthy_needs['Carbs']) &
    (weeks_df['tot_lipides'] >= healthy_needs['Lipides']) &
    (weeks_df['tot_sucres'] >= healthy_needs['Sucres']) &
    (weeks_df['tot_fibres'] >= healthy_needs['Fibres']) &
    (weeks_df['tot_ags'] >= healthy_needs['AGS']) &
    (weeks_df['tot_agm'] >= healthy_needs['AGM']) &
    (weeks_df['tot_agp'] >= healthy_needs['AGP'])
]

In [131]:
healthy_selected_diet['tot_deviation'] = \
    (abs(healthy_selected_diet['tot_energie'] - healthy_needs['Energie'])/healthy_needs['Energie']) + \
    (abs(healthy_selected_diet['tot_proteines'] - healthy_needs['Proteines'])/healthy_needs['Proteines']) + \
    (abs(healthy_selected_diet['tot_carbs'] - healthy_needs['Carbs'])/healthy_needs['Carbs']) + \
    (abs(healthy_selected_diet['tot_lipides'] - healthy_needs['Lipides'])/healthy_needs['Lipides']) + \
    (abs(healthy_selected_diet['tot_sucres'] - healthy_needs['Sucres'])/healthy_needs['Sucres']) + \
    (abs(healthy_selected_diet['tot_fibres'] - healthy_needs['Fibres'])/healthy_needs['Fibres']) + \
    (abs(healthy_selected_diet['tot_ags'] - healthy_needs['AGS'])/healthy_needs['AGS']) + \
    (abs(healthy_selected_diet['tot_agm'] - healthy_needs['AGM'])/healthy_needs['AGM']) + \
    (abs(healthy_selected_diet['tot_agp'] - healthy_needs['AGP'])/healthy_needs['AGP'])

C:\Users\simon\AppData\Local\Temp\ipykernel_24480\1939155408.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  healthy_selected_diet['tot_deviation'] = \


In [132]:
opti_healthy = healthy_selected_diet[healthy_selected_diet['tot_deviation'] == healthy_selected_diet['tot_deviation'].min()]
opti_healthy

,week,tot_energie,tot_proteines,tot_carbs,tot_lipides,tot_sucres,tot_fibres,tot_ags,tot_agm,tot_agp,composition,tot_deviation
3578,3579,43799.03,1149.572,3402.366,2620.397,1002.806,385.086,788.347,1057.503,609.1897,"[[1008, 10025, 20513, 9681, 7720, 39220, 17400...",3460.846225


In [133]:
# Expand the 'composition' column for the first row of opti_healthy into a DataFrame
healthy_opti_plates = pd.DataFrame(opti_healthy.iloc[0]['composition'], columns=required_groups)

# Display the resulting DataFrame
print(healthy_opti_plates.shape)
display(healthy_opti_plates.head())

(21, 7)


,Boissons,Proteines,Vegetables,Grains,Desserts,Diary,Fats
0,1008,10025,20513,9681,7720,39220,17400
1,2016,8015,13004,7110,31102,20911,17700
2,18060,28010,20170,7257,24678,19037,16733
3,18026,34000,4039,9101,24041,19550,16540
4,2027,25232,20513,7425,7745,19037,16150


In [134]:
named_healthy_df = healthy_opti_plates.replace(ciqual_to_name)

named_healthy_df.head(3)

,Boissons,Proteines,Vegetables,Grains,Desserts,Diary,Fats
0,Vodka,"Crabe ou Tourteau, bouilli/cuit à l'eau","Haricot flageolet, bouilli/cuit à l'eau",Graine de couscous (semoule de blé dur précuit...,Pain aux raisins (viennoiserie),"Liégeois aux fruits, préemballé",Huile de sésame
1,"Jus de raisin, pur jus",Rillettes du Mans,"Avocat, pulpe, cru",Pain complet ou intégral (à la farine T150),"Barre céréalière ""équilibre"" chocolatée, enric...","Dessert au soja, aromatisé, sucré, enrichi en ...",Huile combinée (mélange d'huiles)
2,"Cola, non sucré, avec édulcorants","Porc, épaule, cuite","Tomate, purée, appertisée","Muffin anglais, petit pain spécial, préemballé","Biscuit sec (génoise) nappage aux fruits, type...","Lait demi-écrémé, UHT, enrichi en vitamine D s...","Matière grasse végétale (type margarine), à ta..."


In [135]:
cheap_selected_diet = weeks_df[
    (weeks_df['tot_energie'] >= cheap_needs['Energie']) &
    (weeks_df['tot_proteines'] >= cheap_needs['Proteines']) &
    (weeks_df['tot_carbs'] >= cheap_needs['Carbs']) &
    (weeks_df['tot_lipides'] >= cheap_needs['Lipides']) &
    (weeks_df['tot_sucres'] >= cheap_needs['Sucres']) &
    (weeks_df['tot_fibres'] >= cheap_needs['Fibres']) &
    (weeks_df['tot_ags'] >= cheap_needs['AGS']) &
    (weeks_df['tot_agm'] >= cheap_needs['AGM']) &
    (weeks_df['tot_agp'] >= cheap_needs['AGP'])
]

In [136]:
cheap_selected_diet['tot_deviation'] = \
    (abs(cheap_selected_diet['tot_energie'] - cheap_needs['Energie'])/cheap_needs['Energie']) + \
    (abs(cheap_selected_diet['tot_proteines'] - cheap_needs['Proteines'])/cheap_needs['Proteines']) + \
    (abs(cheap_selected_diet['tot_carbs'] - cheap_needs['Carbs'])/cheap_needs['Carbs']) + \
    (abs(cheap_selected_diet['tot_lipides'] - cheap_needs['Lipides'])/cheap_needs['Lipides']) + \
    (abs(cheap_selected_diet['tot_sucres'] - cheap_needs['Sucres'])/cheap_needs['Sucres']) + \
    (abs(cheap_selected_diet['tot_fibres'] - cheap_needs['Fibres'])/cheap_needs['Fibres']) + \
    (abs(cheap_selected_diet['tot_ags'] - cheap_needs['AGS'])/cheap_needs['AGS']) + \
    (abs(cheap_selected_diet['tot_agm'] - cheap_needs['AGM'])/cheap_needs['AGM']) + \
    (abs(cheap_selected_diet['tot_agp'] - cheap_needs['AGP'])/cheap_needs['AGP'])

In [137]:
opti_cheap = cheap_selected_diet[cheap_selected_diet['tot_deviation'] == cheap_selected_diet['tot_deviation'].min()]
opti_cheap

,week,tot_energie,tot_proteines,tot_carbs,tot_lipides,tot_sucres,tot_fibres,tot_ags,tot_agm,tot_agp,composition,tot_deviation
1265,1266,42826.37,1008.283,3041.036,2738.918,1114.526,278.827,1095.427,1090.5539,377.8411,"[[18068, 6531, 4039, 9871, 32000, 19575, 16560...",32.154494


In [138]:
# Expand the 'composition' column for the first row of opti_healthy into a DataFrame
cheap_opti_plates = pd.DataFrame(opti_cheap.iloc[0]['composition'], columns=required_groups)

# Display the resulting DataFrame
print(cheap_opti_plates.shape)
display(cheap_opti_plates.head())

(21, 7)


,Boissons,Proteines,Vegetables,Grains,Desserts,Diary,Fats
0,18068,6531,4039,9871,32000,19575,16560
1,18060,6140,20500,7815,31002,23474,16540
2,1003,21517,13004,7300,39502,19689,17210
3,1004,8232,20258,7210,7745,23474,17701
4,1015,26232,20116,7257,23880,19051,16733


In [139]:
named_cheap_df = cheap_opti_plates.replace(ciqual_to_name)

named_cheap_df.head(3)

,Boissons,Proteines,Vegetables,Grains,Desserts,Diary,Fats
0,"Cola, non sucré, avec édulcorants, sans caféine","Veau, filet, rôti/cuit au four",Rostis ou Galette de pomme de terre,"Pâtes sèches, au blé complet, cuites, non salées","Grains de blé soufflés au miel ou caramel, enr...","Yaourt, lait fermenté ou spécialité laitière, ...",Graisse d'oie
1,"Cola, non sucré, avec édulcorants","Boeuf, joue, crue","Fève, bouillie/cuite à l'eau","Tortilla souple (à garnir), à base de blé","Barre à la noix de coco, enrobée de chocolat",Profiteroles (crème pâtissière et sauce chocol...,Graisse de poulet
2,Liqueur,"Agneau, côte première, crue","Avocat, pulpe, cru",Biscotte classique,Dessert glacé type mystère ou vacherin,"Cheesecake ou Gâteau au fromage frais, préemballé",Huile de noisette


## Local download of final dishes (CIQUAL numbers) 

In [140]:
healthy_opti_plates.to_excel('C:/Users/simon/OneDrive/Bureau/healthy_opti_plates.xlsx', index=False)
cheap_opti_plates.to_excel('C:/Users/simon/OneDrive/Bureau/cheap_opti_plates.xlsx', index=False)